ใน ep ก่อน ๆ ที่เราสร้างโมเดลมา จะเป็นการ Classification คือให้โมเดลจำแนกหมวดหมู่ ผลการทำนายของโมเดล ก็จะถูกจำกัดอยู่ภายใน ตัวเลือก Classes ที่เรากำหนด เช่น A, B, C, D, หมา, แมว, ป่าไม้, แม่น้ำ 

แต่ในเคสนี้ เราจะสร้างโมเดลแบบ Regression คือให้ตอบเป็นค่าอะไรก็ได้ เช่น 5.555555e5, 123456, -0.0000000002865 แต่ใน Dataset นี้คือ ตำแหน่งพิกัด x, y ของจุดศูนย์กลางของหัว

# 0. Magic Commands


In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

# 1. Import Library

In [2]:
from fastai import *
from fastai.vision import *

# 2. ข้อมูล

ใช้ชุดข้อมูล [BIWI head pose dataset](https://data.vision.ee.ethz.ch/cvl/gfanelli/head_pose/head_forest.html#db) label คือตำแหน่งจุดศูนย์กลางของหัว ในแต่ละภาพ. Credit: Gabriele Fanelli et al.

In [3]:
path = untar_data(URLs.BIWI_HEAD_POSE)
path

PosixPath('/home/jupyter/.fastai/data/biwi_head_pose')

แบ่งเป็น Folder training และ testing

In [4]:
path.ls()

[PosixPath('/home/jupyter/.fastai/data/biwi_head_pose/20.obj'),
 PosixPath('/home/jupyter/.fastai/data/biwi_head_pose/03'),
 PosixPath('/home/jupyter/.fastai/data/biwi_head_pose/09.obj'),
 PosixPath('/home/jupyter/.fastai/data/biwi_head_pose/16.obj'),
 PosixPath('/home/jupyter/.fastai/data/biwi_head_pose/23.obj'),
 PosixPath('/home/jupyter/.fastai/data/biwi_head_pose/18'),
 PosixPath('/home/jupyter/.fastai/data/biwi_head_pose/02.obj'),
 PosixPath('/home/jupyter/.fastai/data/biwi_head_pose/22.obj'),
 PosixPath('/home/jupyter/.fastai/data/biwi_head_pose/15'),
 PosixPath('/home/jupyter/.fastai/data/biwi_head_pose/11'),
 PosixPath('/home/jupyter/.fastai/data/biwi_head_pose/08'),
 PosixPath('/home/jupyter/.fastai/data/biwi_head_pose/22'),
 PosixPath('/home/jupyter/.fastai/data/biwi_head_pose/20'),
 PosixPath('/home/jupyter/.fastai/data/biwi_head_pose/24.obj'),
 PosixPath('/home/jupyter/.fastai/data/biwi_head_pose/06.obj'),
 PosixPath('/home/jupyter/.fastai/data/biwi_head_pose/02'),
 PosixPa

ดู Dataset จะเห็นว่ามี Folder แยกตามเลขไว้

ลองดู 20 ไฟล์แรก จาก 1000 ไฟล์ ใน Folder 01

In [24]:
filenames = get_files(path/'01')
len(filenames), filenames[:20]

(1000,
 [PosixPath('/home/jupyter/.fastai/data/biwi_head_pose/01/frame_00294_rgb.jpg'),
  PosixPath('/home/jupyter/.fastai/data/biwi_head_pose/01/frame_00244_pose.txt'),
  PosixPath('/home/jupyter/.fastai/data/biwi_head_pose/01/frame_00313_rgb.jpg'),
  PosixPath('/home/jupyter/.fastai/data/biwi_head_pose/01/frame_00089_rgb.jpg'),
  PosixPath('/home/jupyter/.fastai/data/biwi_head_pose/01/frame_00214_pose.txt'),
  PosixPath('/home/jupyter/.fastai/data/biwi_head_pose/01/frame_00474_rgb.jpg'),
  PosixPath('/home/jupyter/.fastai/data/biwi_head_pose/01/frame_00438_pose.txt'),
  PosixPath('/home/jupyter/.fastai/data/biwi_head_pose/01/frame_00434_rgb.jpg'),
  PosixPath('/home/jupyter/.fastai/data/biwi_head_pose/01/frame_00117_pose.txt'),
  PosixPath('/home/jupyter/.fastai/data/biwi_head_pose/01/frame_00410_rgb.jpg'),
  PosixPath('/home/jupyter/.fastai/data/biwi_head_pose/01/frame_00340_pose.txt'),
  PosixPath('/home/jupyter/.fastai/data/biwi_head_pose/01/frame_00142_pose.txt'),
  PosixPath('/h

โครงสร้าง Folder จะเป็นดังนี้

- 01.obj
- 01
    - rgb.cal
    - frame_00001_pose.txt
    - frame_00001_rgb.txt
    - frame_00002_pose.txt
    - frame_00002_rgb.txt
    - frame_00003_pose.txt
    - frame_00003_rgb.txt
    - ...
- 02.obj
- 02
    - rgb.cal
    - fname_...


In [ ]:
cal = np.genfromtxt(path/'01'/'rgb.cal', skip_footer=6); cal

In [ ]:
fname = '09/frame_00667_rgb.jpg'

In [ ]:
def img2txt_name(f): return path/f'{str(f)[:-7]}pose.txt'

In [ ]:
img = open_image(path/fname)
img.show()

In [ ]:
ctr = np.genfromtxt(img2txt_name(fname), skip_header=3); ctr

In [ ]:
def convert_biwi(coords):
    c1 = coords[0] * cal[0][0]/coords[2] + cal[0][2]
    c2 = coords[1] * cal[1][1]/coords[2] + cal[1][2]
    return tensor([c2,c1])

def get_ctr(f):
    ctr = np.genfromtxt(img2txt_name(f), skip_header=3)
    return convert_biwi(ctr)

def get_ip(img,pts): return ImagePoints(FlowField(img.size, pts), scale=True)

In [ ]:
get_ctr(fname)

In [ ]:
ctr = get_ctr(fname)
img.show(y=get_ip(img, ctr), figsize=(6, 6))

# 3. เตรียมข้อมูล



1. สำหรับข้อมูลประเภทตัวเลข เราไม่สามารถ flip ซ้าย ขวา ขึ้น ลง ได้ จะทำให้ผิดความหมายไป จึงต้องใช้ get_transforms(do_flip=False)
2. เคสนี้ label คือชื่อ Folder ที่บรรจุไฟล์อยู่ เราสามารถ ใช้ ImageDataBunch.from_folder ได้เลย
3. num_workers คือ จำนวน CPU Core ที่ต้องการใช้

In [ ]:
batchsize = 64
np.random.seed(0)
transform = get_transforms()

databunch = (PointsItemList.from_folder(path)
        .split_by_valid_func(lambda o: o.parent.name=='13')
        .label_from_func(get_ctr)
        .transform(transform, tfm_y=True, size=(120,160))
        .databunch(bs=batchsize).normalize(imagenet_stats)
       )

## สำรวจข้อมูล ##

ลองดูข้อมูล Batch แรก ด้วย show_batch สั่งให้แสดง รูป พร้อม label ไว้บนรูป แสดง 3 แถว (rows) ในกรอบขนาด 10 x 10 นิ้ว (figsize)

*เราสามารถรัน cell นี้หลายครั้ง เพื่อเรียกดู batch ต่อ ๆ ไป ได้เรื่อย ๆ เป็นการสำรวจข้อมูล*

In [ ]:
databunch.show_batch(rows=3, figsize=(10, 10))

ดู label ตัวเลขทั้งหมด หรือ class ทั้งหมด ที่มีใน Dataset

In [ ]:
databunch.classes

# 4. สร้างโมเดล

งานไม่ค่อยซับซ้อน เราจะใช้ resnet34

In [ ]:
learner = cnn_learner(databunch, models.resnet34, 
                     callback_fns=ShowGraph)

# 5. เริ่มต้นเทรนโมเดล

เทรนด้วยค่า Default ไป 4 Cycle

In [ ]:
learner.fit_one_cycle(4)

# 6. เทรนต่อ

In [ ]:
learner.unfreeze()

In [ ]:
learner.recorder.plot()

In [ ]:
learner.fit_one_cycle(3, max_lr=slice(1e-6, 3e-3))

## สำเร็จแล้ว ##

เพียงแค่เวลา x นาที เราเทรน Model ได้ accuracy ประมาณ x หรือ ความแม่นยำประมาณ x


# 7. ดูผลลัพธ์

In [ ]:
learner.show_results(rows=3)

In [ ]:
interpretation = ClassificationInterpretation.from_learner(learner)

แสดงรายการที่โมเดลทายผิด 9 อันดับแรก


In [ ]:
interpretation.plot_top_losses(9)

จะเห็นได้ว่า

# 9. สรุป


1. เราสามารถสร้างโมเดล Regression ได้ง่าย ๆ ด้วยชุดข้อมูลรูปภาพ, โค้ดไม่กี่บรรทัด และ GPU ไม่กี่นาที
2. แต่อาจจะยังงงอยู่ ว่ามันทำงานอย่างไร ไว้เราจะค่อย ๆ อธิบายต่อไป

# Credit 

* [FastAI: Practical Deep Learning for Coders, v3 - Lesson 3](https://course.fast.ai/videos/?lesson=3)
* [Biwi Kinect Head Pose Database](https://data.vision.ee.ethz.ch/cvl/gfanelli/head_pose/head_forest.html#db)